<h1> Classification: </h1>
<p> In ML, classification is the task of reciving input data, and determining the probabilities that it belongs to a set of output groups.</p>
<p> Less abstractly, a good example of this would be: say you have a bunch of pictures of animals. You want to determine which animal is in each picture. You set up a bunch of categories beforehand (i.e dog, cat, bird) and then you train a model to assign probabilities to each category given an input picture. Then, if you can train the model to have a high accuracy, you can get it to "guess" which animal is in each picture, classifying the picture with high accuracy. </p>
<p> For this example, I'm going to be using the iris-flower dataset, which has a bunch of measurements of flower characteristics, as well as a "classification" of what species that flower is. I will try to train a model to predict that classification with high accuracy. </p>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import pandas as pd

<h2> Dataset: </h2>

This specific dataset seperates flowers into 3 different classes of species.
- Setosa
- Versicolor
- Virginica

The information about each flower is the following.
- sepal length
- sepal width
- petal length
- petal width

In [5]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [6]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [7]:
#can look at our training data
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [8]:
#and our testing data
test.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


<p> Here our encoding for the "species" classification is as follows: </p>
<ul>
    <li> Setosa: 0 </li>
    <li> Versicolor: 1 </li>
    <li> Virginica: 2 </li>
</ul>

In [9]:
#Use the species column as the label (category)
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [11]:
#make our input function
#simpler because no epochs
def input_fn(features, labels, training=True, batch_size=256):
    #convert input to tensorflow dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
    #shuffle the dataset if training the model
    if training:
        dataset = dataset.shuffle(1000).repeat()
        
    return dataset.batch(batch_size)

In [12]:
#make our feature columns
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


<p> We can see the different features stored nicely
in the my_feature_columns array. Now we will choose our model. We use a deep-neural network classifier with two hidden layers, of 30 and 10 nodes each.</p>

In [18]:
classifier = tf.estimator.DNNClassifier(
    #features
    feature_columns=my_feature_columns,
    #layers
    hidden_units=[100,100,100],
    #3 choices for classification
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/rf/q21h0s5d18j5rw4wyg_lh5x00000gn/T/tmp2mc8rw9h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
#train model
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=3800)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/rf/q21h0s5d18j5rw4wyg_lh5x00000gn/T/tmp2mc8rw9h/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/rf/q21h0s5d18j5rw4wyg_lh5x00000gn/T/tmp2mc8rw9h/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.079852596, step = 5000
INFO:tensorflow:global_step/sec: 175.196
INFO:tensorflow:loss = 0.08178504, step = 5100 (0.573 sec)
INFO:tensorflow:global_step/sec: 315.84
INFO:tensorflow:loss = 0.082440875, step = 5200 (0.315 sec)
INFO:tensorflow:global_step/sec: 224.465
INFO:ten

In [24]:
#evaluate model
eval_result=classifier.evaluate(input_fn=lambda: input_fn(test,test_y, training=False))
print("accuracy: {}".format(eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-22T18:42:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/rf/q21h0s5d18j5rw4wyg_lh5x00000gn/T/tmp2mc8rw9h/model.ckpt-8800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.31715s
INFO:tensorflow:Finished evaluation at 2021-10-22-18:42:35
INFO:tensorflow:Saving dict for global step 8800: accuracy = 0.96666664, average_loss = 0.06386842, global_step = 8800, loss = 0.06386842
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8800: /var/folders/rf/q21h0s5d18j5rw4wyg_lh5x00000gn/T/tmp2mc8rw9h/model.ckpt-8800
accuracy: {'accuracy': 0.96666664, 'average_loss': 0.06386842, 'loss': 0.06386842, 'global_step': 8800}
